In [145]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
import tensorflow as tf
import numpy as np
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Custom Loss Function

In [146]:
def huber_fn(y_true,y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error)<1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error)-.5
    return tf.where(is_small_error,squared_loss,linear_loss)
def create_huber(threshold=1.0):
    def huber_fn(y_true,y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error)<threshold
        squared_loss = tf.square(error)/2
        linear_loss = threshold * tf.abs(error) - threshold**2/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    return huber_fn

In [147]:
X_train[1,:].shape

(8,)

In [148]:
input_shape = X_train[1,:].shape
model = keras.models.Sequential([
    keras.layers.Dense(30,activation = "selu", kernel_initializer = "lecun_normal",
                      input_shape=input_shape),
    keras.layers.Dense(1)
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_78 (Dense)             (None, 30)                270       
_________________________________________________________________
dense_79 (Dense)             (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [149]:
model.compile(loss=huber_fn,optimizer="nadam",metrics=["mae"])
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))

Epoch 1/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5835 - mae: 0.9320 - val_loss: 0.3595 - val_mae: 0.6503
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2174 - mae: 0.5141 - val_loss: 0.3124 - val_mae: 0.6027
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2084 - mae: 0.5015 - val_loss: 0.2689 - val_mae: 0.5527
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2019 - mae: 0.4929 - val_loss: 0.2290 - val_mae: 0.5124
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1965 - mae: 0.4853 - val_loss: 0.1911 - val_mae: 0.4716
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1919 - mae: 0.4784 - val_loss: 0.1794 - val_mae: 0.4585
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1903 - mae: 0.4770 - val_loss: 0.1889 - val_mae: 0.4671
Epoch 8/10
363/363 [==============================] - 0s 1ms/step - loss: 0.

In [150]:
model.save("../Models/5_CustomModelAndTraining")

INFO:tensorflow:Assets written to: ../Models/5_CustomModelAndTraining/assets


In [151]:
model = keras.models.load_model("../Models/5_CustomModelAndTraining",
                                custom_objects={"huber_fn":huber_fn})

In [152]:
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))

Epoch 1/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1832 - mean_absolute_error: 0.4649 - val_loss: 0.1904 - val_mean_absolute_error: 0.4650
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1819 - mean_absolute_error: 0.4626 - val_loss: 0.1717 - val_mean_absolute_error: 0.4493
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1804 - mean_absolute_error: 0.4605 - val_loss: 0.1836 - val_mean_absolute_error: 0.4572
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1788 - mean_absolute_error: 0.4578 - val_loss: 0.1723 - val_mean_absolute_error: 0.4462
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1778 - mean_absolute_error: 0.4561 - val_loss: 0.1879 - val_mean_absolute_error: 0.4597
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1764 - mean_absolute_error: 0.4535 - val_loss: 0.1710 - val_mean_absolute_error: 0.4422
Epoch 7/10
363/3

# Custom Loss Class
### Threshold is saved with saving of model

In [153]:
class HuberLoss(keras.losses.Loss):
    def __init__(self,threshold = 1.0,**kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self,y_true,y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error)<self.threshold
        squared_loss = tf.square(error)/2
        linear_loss = self.threshold* tf.abs(error)-self.threshold**2/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return{**base_config,"threshold":self.threshold}

In [154]:
model.compile(loss=HuberLoss(2.0),optimizer="nadam",metrics=["mae"])
model.save("../Models/5_CustomModelAndTraining_HuberClass")

INFO:tensorflow:Assets written to: ../Models/5_CustomModelAndTraining_HuberClass/assets


In [155]:
model = keras.models.load_model("../Models/5_CustomModelAndTraining_HuberClass/",
                               custom_objects={"HuberLoss":HuberLoss})

In [156]:
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))

Epoch 1/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1909 - mean_absolute_error: 0.4478 - val_loss: 0.2263 - val_mean_absolute_error: 0.4543
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1904 - mean_absolute_error: 0.4463 - val_loss: 0.1915 - val_mean_absolute_error: 0.4409
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1879 - mean_absolute_error: 0.4441 - val_loss: 0.2135 - val_mean_absolute_error: 0.4465
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1872 - mean_absolute_error: 0.4426 - val_loss: 0.1928 - val_mean_absolute_error: 0.4391
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1865 - mean_absolute_error: 0.4414 - val_loss: 0.2180 - val_mean_absolute_error: 0.4497
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1855 - mean_absolute_error: 0.4404 - val_loss: 0.1863 - val_mean_absolute_error: 0.4353
Epoch 7/10
363/3

## Custom Activation, Initializer,Reularizer Functions

In [157]:
def my_softplus(z):
    return tf.math.log(tf.exp(z)+1.0)
def my_glorot_initializer(shape,dtype=tf.float32):
    stddev = tf.sqrt(2./(shape[0]+shape[1]))
    return tf.random.normal(shape,stddev=stddev,dtype=dtype)
def my_li_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01*weights))
def my_positive_weights(weights):
    return tf.where(weights<0.,tf.zeros_like(weights),weights)

In [158]:
layer = keras.layers.Dense(30,activation = my_softplus,
                          kernel_initializer = my_glorot_initializer,
                          kernel_regularizer=my_li_regularizer,
                          kernel_constraint = my_positive_weights)

In [159]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self,factor):
        self.factor = factor
    def __call__(self,weights):
        return tf.reduce_sum(tf.abs(self.factor*weights))
    def get_config(self):
        return {"factor":factor}

In [160]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self,threshold=1.0,**kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total",initializer="zeros")
        self.count = self.add_weight("count",initializer="zeros")
    def update_state(self,y_true,y_pred,sample_weight=None):
        metric = self.huber_fn(y_true,y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true),tf.float32))
    def result(self):
        return self.total/self.count
    def get_config(self):
        base_config = super().get_config()
        return{**base_config,"threshold":threshold}

In [161]:
#Better Implementation
class HuberMetric(keras.metrics.Mean):
    def __init__(self, threshold=1.0, name='HuberMetric', dtype=None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        super(HuberMetric, self).update_state(metric, sample_weight)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [162]:
model.compile(loss=keras.losses.Huber(2.0), optimizer="nadam", weighted_metrics=[HuberMetric(2.0)])


In [163]:
sample_weight = np.random.rand(len(y_train))
history = model.fit(X_train_scaled.astype(np.float32), y_train.astype(np.float32),
                    epochs=2, sample_weight=sample_weight)

Epoch 1/2
363/363 [==============================] - 0s 1ms/step - loss: 0.0923 - HuberMetric: 0.1846
Epoch 2/2
363/363 [==============================] - 0s 886us/step - loss: 0.0924 - HuberMetric: 0.1847


In [164]:
history.history["loss"][0], history.history["HuberMetric"][0] * sample_weight.mean()
model.save("../Models/5_ModelWithCustomMetric")

INFO:tensorflow:Assets written to: ../Models/5_ModelWithCustomMetric/assets


In [165]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)} 

In [166]:
model = keras.models.Sequential([MyDense(30, activation = 'relu', input_shape=input_shape),
                                MyDense(1)])

In [167]:
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 0s 1ms/step - loss: 1.5663 - val_loss: 1.0096
Epoch 2/2
162/162 [==============================] - 0s 696us/step - loss: 0.5548


0.5548022389411926

In [168]:
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self,stddev,**kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
    def call(self,X,training = None):
        if training:
            noise = tf.random.normal(tf.shape(X),stddev=self.stddev)
            return noise + X
        else:
            return X
    def compute_output_shape(self,batch_input_shape):
        return batch_input_shape
        

# Custom Layers and Models

In [169]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self,n_layers,n_neurons,**kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons,activation="relu",
                                         kernel_initializer = "he_normal")
                      for _ in range(n_layers)]
    def call(self,inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [170]:
class ResidualRegressor(keras.Model):
    def __init__(self,output_dim,**kwags):
        super().__init__(**kwags)
        self.hidden1 = keras.layers.Dense(30,activation="elu",
                                          kernel_initializer = "he_normal")
        self.block1 = ResidualBlock(2,30)
        self.block2 = ResidualBlock(2,30)
        self.out = keras.layers.Dense(output_dim)
    def call(self, inputs):
        Z= self.hidden1(inputs)
        for _ in range (1+3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

In [171]:
X_new_scaled = X_test_scaled

model = ResidualRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5)
score = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_new_scaled)
model.summary()

Epoch 1/5
363/363 [==============================] - 0s 1ms/step - loss: 30.5819
Epoch 2/5
363/363 [==============================] - 0s 1ms/step - loss: 2.5190
Epoch 3/5
363/363 [==============================] - 0s 1ms/step - loss: 1.7493
Epoch 4/5
363/363 [==============================] - 0s 1ms/step - loss: 1.7732
Epoch 5/5
162/162 [==============================] - 0s 778us/step - loss: 0.8806
Model: "residual_regressor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             multiple                  270       
_________________________________________________________________
residual_block (ResidualBloc multiple                  1860      
_________________________________________________________________
residual_block_1 (ResidualBl multiple                  1860      
_________________________________________________________________
dense_86 (Dense)             multiple    

In [172]:
model.save("../Models/5_CustomLayerModel")

INFO:tensorflow:Assets written to: ../Models/5_CustomLayerModel/assets


In [173]:

class ReconstructingRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                          kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        # TODO: check https://github.com/tensorflow/tensorflow/issues/26260
        #self.reconstruction_mean = keras.metrics.Mean(name="reconstruction_error")

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        self.recon_loss = 0.05 * tf.reduce_mean(tf.square(reconstruction - inputs))
        return self.out(Z)
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x)
            loss = self.compiled_loss(y, y_pred, regularization_losses=[self.recon_loss])

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return {m.name: m.result() for m in self.metrics}

In [174]:
keras.losses.Loss()
model2 = ReconstructingRegressor(1)
model2.compile(loss="mse", optimizer="nadam")
history = model2.fit(X_train_scaled, y_train, epochs=2)
#y_pred = model.predict(X_test_scaled)

Epoch 1/2
363/363 [==============================] - 0s 999us/step - loss: 0.6870
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.4027
